# Transformer Decoder

Building the Decoder Block and the full Decoder Stack (GPT Style).


In [ ]:
import torch
import torch.nn as nn
from typing import Optional, List, Tuple


In [ ]:
class DecoderBlock(nn.Module):
    """
    Single Transformer Decoder Block.
    
    Implements the decoder layer with:
    - Masked self-attention (causal, for autoregressive generation)
    - Optional cross-attention to encoder output (for encoder-decoder models)
    - Feed-forward network
    - Residual connections and layer normalization
    
    For GPT-style models, set use_cross_attention=False.
    
    Args:
        d_model: Dimension of the model
        n_heads: Number of attention heads
        d_ff: Dimension of the feed-forward hidden layer
        dropout: Dropout probability
        activation: Activation function for FFN
        pre_norm: Whether to use pre-layer normalization
        use_cross_attention: Whether to include cross-attention layer
        
    Example:
        >>> # GPT-style (decoder-only)
        >>> block = DecoderBlock(d_model=512, n_heads=8, d_ff=2048, use_cross_attention=False)
        >>> x = torch.randn(2, 10, 512)
        >>> output, self_attn, _ = block(x)
        
        >>> # Encoder-decoder style
        >>> block = DecoderBlock(d_model=512, n_heads=8, d_ff=2048, use_cross_attention=True)
        >>> enc_output = torch.randn(2, 20, 512)
        >>> output, self_attn, cross_attn = block(x, encoder_output=enc_output)
    """
    
    def __init__(
        self,
        d_model: int,
        n_heads: int,
        d_ff: int,
        dropout: float = 0.1,
        activation: str = "gelu",
        pre_norm: bool = True,
        layer_norm_eps: float = 1e-6,
        use_cross_attention: bool = False
    ):
        super().__init__()
        
        self.pre_norm = pre_norm
        self.use_cross_attention = use_cross_attention
        
        # Masked multi-head self-attention
        self.self_attention = MultiHeadAttention(d_model, n_heads, dropout)
        
        # Cross-attention (optional, for encoder-decoder models)
        if use_cross_attention:
            self.cross_attention = MultiHeadAttention(d_model, n_heads, dropout)
            self.norm_cross = nn.LayerNorm(d_model, eps=layer_norm_eps)
        
        # Feed-forward network
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout, activation)
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        
        # Dropout for residual connections
        self.dropout = nn.Dropout(dropout)
        
    def forward(
        self,
        x: torch.Tensor,
        encoder_output: Optional[torch.Tensor] = None,
        self_attn_mask: Optional[torch.Tensor] = None,
        cross_attn_mask: Optional[torch.Tensor] = None,
        return_attention: bool = False
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[torch.Tensor]]:
        """
        Forward pass through the decoder block.
        
        Args:
            x: Input tensor of shape (batch, seq, d_model)
            encoder_output: Encoder output for cross-attention (batch, enc_seq, d_model)
            self_attn_mask: Mask for self-attention (causal mask)
            cross_attn_mask: Mask for cross-attention (padding mask)
            return_attention: Whether to return attention weights
            
        Returns:
            Tuple of:
                - Output tensor of shape (batch, seq, d_model)
                - Self-attention weights if return_attention=True
                - Cross-attention weights if return_attention=True and use_cross_attention=True
        """
        self_attn_weights = None
        cross_attn_weights = None
        
        if self.pre_norm:
            # Pre-Layer Normalization
            
            # 1. Masked Self-Attention
            attn_out, self_attn_weights = self._self_attention_block(
                self.norm1(x), self_attn_mask, return_attention
            )
            x = x + attn_out
            
            # 2. Cross-Attention (optional)
            if self.use_cross_attention and encoder_output is not None:
                cross_out, cross_attn_weights = self._cross_attention_block(
                    self.norm_cross(x), encoder_output, cross_attn_mask, return_attention
                )
                x = x + cross_out
            
            # 3. Feed-Forward
            x = x + self._ff_block(self.norm2(x))
            
        else:
            # Post-Layer Normalization
            
            # 1. Masked Self-Attention
            attn_out, self_attn_weights = self._self_attention_block(
                x, self_attn_mask, return_attention
            )
            x = self.norm1(x + attn_out)
            
            # 2. Cross-Attention (optional)
            if self.use_cross_attention and encoder_output is not None:
                cross_out, cross_attn_weights = self._cross_attention_block(
                    x, encoder_output, cross_attn_mask, return_attention
                )
                x = self.norm_cross(x + cross_out)
            
            # 3. Feed-Forward
            x = self.norm2(x + self._ff_block(x))
            
        return x, self_attn_weights, cross_attn_weights
    
    def _self_attention_block(
        self,
        x: torch.Tensor,
        mask: Optional[torch.Tensor],
        return_attention: bool
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """Apply masked self-attention with dropout."""
        attn_output, attn_weights = self.self_attention(
            x, x, x, mask=mask, return_attention=return_attention
        )
        return self.dropout(attn_output), attn_weights
    
    def _cross_attention_block(
        self,
        x: torch.Tensor,
        encoder_output: torch.Tensor,
        mask: Optional[torch.Tensor],
        return_attention: bool
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """Apply cross-attention to encoder output with dropout."""
        attn_output, attn_weights = self.cross_attention(
            query=x,
            key=encoder_output,
            value=encoder_output,
            mask=mask,
            return_attention=return_attention
        )
        return self.dropout(attn_output), attn_weights
    
    def _ff_block(self, x: torch.Tensor) -> torch.Tensor:
        """Apply feed-forward network with dropout."""
        return self.dropout(self.feed_forward(x))


In [ ]:
class Decoder(nn.Module):
    """
    Transformer Decoder Stack.
    
    Combines embedding layer with multiple decoder blocks to create
    the complete decoder for autoregressive text generation.
    
    For GPT-style models, this is the entire model (no encoder).
    
    Args:
        vocab_size: Size of the vocabulary
        d_model: Dimension of the model
        n_heads: Number of attention heads
        n_layers: Number of decoder blocks
        d_ff: Dimension of the feed-forward hidden layer
        max_seq_len: Maximum sequence length
        dropout: Dropout probability
        activation: Activation function for FFN
        pre_norm: Whether to use pre-layer normalization
        use_cross_attention: Whether decoder blocks include cross-attention
        
    Example:
        >>> # GPT-style decoder (no encoder)
        >>> decoder = Decoder(vocab_size=10000, d_model=512, n_heads=8, n_layers=6, d_ff=2048)
        >>> tokens = torch.randint(0, 10000, (2, 50))
        >>> logits = decoder(tokens)
        >>> print(logits.shape)  # (2, 50, 10000)
    """
    
    def __init__(
        self,
        vocab_size: int,
        d_model: int,
        n_heads: int,
        n_layers: int,
        d_ff: int,
        max_seq_len: int = 512,
        dropout: float = 0.1,
        activation: str = "gelu",
        pre_norm: bool = True,
        layer_norm_eps: float = 1e-6,
        use_cross_attention: bool = False
    ):
        super().__init__()
        
        self.d_model = d_model
        self.n_layers = n_layers
        self.max_seq_len = max_seq_len
        
        # Embedding layer (token + positional)
        self.embedding = TransformerEmbedding(
            vocab_size=vocab_size,
            d_model=d_model,
            max_seq_len=max_seq_len,
            dropout=dropout
        )
        
        # Stack of decoder blocks
        self.layers = nn.ModuleList([
            DecoderBlock(
                d_model=d_model,
                n_heads=n_heads,
                d_ff=d_ff,
                dropout=dropout,
                activation=activation,
                pre_norm=pre_norm,
                layer_norm_eps=layer_norm_eps,
                use_cross_attention=use_cross_attention
            )
            for _ in range(n_layers)
        ])
        
        # Final layer normalization (for pre-norm)
        self.final_norm = nn.LayerNorm(d_model, eps=layer_norm_eps) if pre_norm else None
        
        # Output projection to vocabulary
        self.output_projection = nn.Linear(d_model, vocab_size, bias=False)
        
        # Pre-compute causal mask
        self._register_causal_mask(max_seq_len)
        
        # Initialize weights
        self._init_weights()
        
    def _register_causal_mask(self, max_seq_len: int):
        """Pre-compute and register the causal attention mask."""
        # Create upper triangular mask (True = masked)
        mask = torch.triu(torch.ones(max_seq_len, max_seq_len), diagonal=1).bool()
        # Shape: (1, 1, max_seq_len, max_seq_len) for broadcasting
        self.register_buffer('causal_mask', mask.unsqueeze(0).unsqueeze(0))
        
    def _init_weights(self):
        """Initialize weights with small values for stable training."""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(
        self,
        tgt: torch.Tensor,
        encoder_output: Optional[torch.Tensor] = None,
        cross_attn_mask: Optional[torch.Tensor] = None,
        return_attention: bool = False
    ) -> Tuple[torch.Tensor, Optional[List[Tuple]]]:
        """
        Decode input sequence autoregressively.
        
        Args:
            tgt: Target token indices of shape (batch, seq)
            encoder_output: Optional encoder output for cross-attention
            cross_attn_mask: Optional mask for cross-attention
            return_attention: Whether to return attention weights
            
        Returns:
            Tuple of:
                - Logits of shape (batch, seq, vocab_size)
                - List of (self_attn, cross_attn) tuples if return_attention=True
        """
        seq_len = tgt.size(1)
        attention_weights = [] if return_attention else None
        
        # Get the causal mask for the current sequence length
        causal_mask = self.causal_mask[:, :, :seq_len, :seq_len]
        
        # Apply embeddings
        x = self.embedding(tgt)
        
        # Pass through decoder blocks
        for layer in self.layers:
            x, self_attn, cross_attn = layer(
                x,
                encoder_output=encoder_output,
                self_attn_mask=causal_mask,
                cross_attn_mask=cross_attn_mask,
                return_attention=return_attention
            )
            if return_attention:
                attention_weights.append((self_attn, cross_attn))
        
        # Apply final layer normalization
        if self.final_norm is not None:
            x = self.final_norm(x)
        
        # Project to vocabulary
        logits = self.output_projection(x)
        
        return logits, attention_weights
    
    def get_embedding(self, tgt: torch.Tensor) -> torch.Tensor:
        """Get just the embeddings without passing through layers."""
        return self.embedding(tgt)
